## Trajectory Inference Tutorial 4

In [ ]:
suppressMessages(library(velocyto.R))
library(tools)
library(pagoda2)
library(RColorBrewer)
library(CytoTRACE)
options(USE_OPENMP =1)

### Data Import

In [ ]:
ldat <- read.loom.matrices("MACA_bonemarrow_10x_subsetted.loom")
phenotable <- read.table("MACA_bonemarrow_10x_metadata.txt", sep = "\t", header = T)
mat <- read.table("MACA_bonemarrow_10x_expr.txt", sep = "\t", row.names =1, header = T)

### RNA velocity
Pagoda2 processing

In [ ]:
emat <- ldat$spliced
rownames(emat) <- make.unique(rownames(emat))
nmat <- ldat$unspliced
# rownames(nmat) <- make.unique(rownames(nmat)
r <- Pagoda2$new(emat, modelType = "plain", trim = 10, log.scale = T)
r$adjustVariance(gam.k = 10)
r$calculatePcaReduction(nPcs = 100, n.odgenes = 3e3, maxit = 300)
r$makeKnnGraph(k = 30, type = "PCA", center = T, distance = "cosine")
r$getKnnClusters(method = multilevel.community, type = "PCA", name = "multilevel")
r$getEmbedding(type = "PCA", embeddingType = "tSNE", perplexity = 50, verbose = T)
cluster.label <- r$clusters$PCA[[1]]
names(cluster.label) <- colnames(emat)
# cell.colors <- pagoda2:::fac2col(cluster.label)
emb <- r$embeddings$PCA$tSNE
cell.dist <- as.dist(1-armaCor(t(r$reductions$PCA)))
emat <- filter.genes.by.cluster.expression(emat,cluster.label,min.max.cluster.average = 0.5)
nmat <- filter.genes.by.cluster.expression(nmat,cluster.label,min.max.cluster.average = 0.05)

### Generate RNA velocity object

In [ ]:
rvel <- gene.relative.velocity.estimates(emat, nmat, deltaT = 1, kCells = 70, cell.dist = cell.dist, fit.quantile = 0.02, n.cores = 4)

### Obtain PCA coordinates

In [ ]:
pcs <- pca.velocity.plot(rvel, plot.cols=1, nPcs=4, cex=1.2, pcount=0.1, arrow.scale=0.5, return.details=F, show.grid.flow=F, n.cores = 4)
emb <- pcs$epc@scores

### CytoTRACE#

In [ ]:
results <- CytoTRACE(mat, enableFast = F)
cytotrace <- results$CytoTRACE

### plot data
#### Setup colors

In [ ]:
rbPal <- colorRampPalette(brewer.pal(11, "Spectral"))
pdf("rnavelocity_Marrow_10x.pdf", width = 17.5, height = 5.25, useDingbats = FALSE)
par(mfrow=c(1,3), mar = c(b=6, l=7, t=2, r=0))

### Phenotype plot

In [ ]:
phenotype <- as.character(unlist(phenotable$order))
col2 <- rbPal(length(unique(phenotype)))
names(col2) <- unique(phenotype[order(phenotable$order)])
col2 <- col2[phenotype]
plot(emb[,1], emb[,2], pch = 19, cex=2, col = ac(col2, 0.5), xlab="", ylab="", las = 1,
     frame.plot = T,
     cex.axis = 2.5, xaxt="n")
axis(1, cex.axis=2.5, padj=0.5)
mtext("PC2", cex=2, side=2, line = 4.5)
mtext("PC1", cex =2, side = 1, line=4.5)
# plotCytoTRACE(results, emb = emb, phenotype = phenotable$phenotype)

### CytoTRACE plot

In [ ]:
par(mar = c(b=6, l=3.5, t=2, r=3.5))
col3 <- rev(rbPal(length(cytotrace)))[as.numeric(cut(cytotrace, breaks = length(cytotrace)))]

plot(emb[,1], emb[,2], pch = 19, cex=2, col = ac(col3, 0.5), xlab="", ylab="", yaxt="n", las = 1,
     frame.plot = T,
     cex.axis = 2.5, xaxt="n")
axis(1, cex.axis=2.5, padj=0.5)
mtext("PC1", cex = 2, side = 1, line=4.5)

### RNA velocity plot

In [ ]:
par(mar = c(b=6, l=0, t=2, r=7))
col4 <- rep("gray82", length(phenotype))
names(col4) <- colnames(rvel$projected)
file <- show.velocity.on.embedding.cor(emb, rvel, n = 300, scale = "sqrt",
                                       cell.colors = ac(col4,0.5), cex = 2, arrow.scale = 10,
                                       n.cores = 4, show.grid.flow = T, min.grid.cell.mass = 0.5,
                                       grid.n = 20, arrow.lwd = 2, do.par = F, xaxt = "n", yaxt = "n",
                                       col.main = "white", col.lab = "white", return.details = T)
axis(1, cex.axis=2.5, padj=0.5)
mtext("PC1", cex = 2, side = 1, line=4.5)
dev.off()